# 現代制御(Modern Control)

## 状態空間表現
$
\begin{cases}
\dot{x}(t)=Ax(t)+Bu(t) \\ 
y(t)=Cx(t)+Du(t)
\end{cases}
$

$x(t)$:状態変数ベクトル(状態ベクトル), $u(t)$:入力ベクトル, $y(t)$:出力ベクトル  
$A$:システム行列(状態行列), $B$:入力係数行列(入力行
列), $C$:出力変数ベクトル(出力行列), $D$:直達行列

※直達行列$D$は基本0なので基本的に出力方程式は$y(t)=Cx(t)$

状態方程式 = 1階の常微分方程式  
→ 行列を用いることで高階の微分方程式も統一した方法で表現可能


### 使用するライブラリ
[Python Control Systems Library](https://python-control.readthedocs.io/en/0.8.3/)

### 参考
+ [状態方程式の解](http://ysserve.wakasato.jp/Lecture/ControlMecha2/node15.html)

In [2]:
from control.matlab import *
import sympy
from sympy import symbols, Symbol, var, init_printing, integrate
init_printing()

## 状態遷移行列
状態遷移行列は下記の式で表される  
$
e^{At}
$  
$A$:状態行列, t:時間

ラプラス変換を用いる場合は下記の等式が成立する  
$
e^{At} = \mathcal{L}^{-1}[(sI-A)^{-1}]
$

In [57]:
A = sympy.Matrix([[0, 1], [-5, -6]])
s = sympy.var('s')
I = sympy.eye(2)
result1 = (s*I - A)
display(result1)
display(result1.inv())
display(sympy.apart(result1.inv()))
t = sympy.var('t')
sympy.exp(A*t)

⎡s   -1  ⎤
⎢        ⎥
⎣5  s + 6⎦

⎡    -5⋅s - 30             -5        ⎤
⎢─────────────────  ─────────────────⎥
⎢-5⋅s⋅(s + 6) - 25  -5⋅s⋅(s + 6) - 25⎥
⎢                                    ⎥
⎢       5                 -s         ⎥
⎢ ──────────────     ──────────────  ⎥
⎣ -s⋅(s + 6) - 5     -s⋅(s + 6) - 5  ⎦

⎡      1           5            1           1    ⎤
⎢- ───────── + ─────────  - ───────── + ─────────⎥
⎢  4⋅(s + 5)   4⋅(s + 1)    4⋅(s + 5)   4⋅(s + 1)⎥
⎢                                                ⎥
⎢     5           5            5           1     ⎥
⎢ ───────── - ─────────    ───────── - ───────── ⎥
⎣ 4⋅(s + 5)   4⋅(s + 1)    4⋅(s + 5)   4⋅(s + 1) ⎦

⎡     -t    -5⋅t       -t    -5⋅t  ⎤
⎢  5⋅ℯ     ℯ          ℯ     ℯ      ⎥
⎢  ───── - ─────      ─── - ─────  ⎥
⎢    4       4         4      4    ⎥
⎢                                  ⎥
⎢     -t      -5⋅t     -t      -5⋅t⎥
⎢  5⋅ℯ     5⋅ℯ        ℯ     5⋅ℯ    ⎥
⎢- ───── + ───────  - ─── + ───────⎥
⎣    4        4        4       4   ⎦

## 状態方程式の一般解
$
x(t) = e^{At}x_0 + \int_0^t e^{A(t-\tau)}Bu(\tau)d\tau
$
### n次システムの時間応答(任意の入力$u(t)$,任意の初期値$x_0$)
$
x(t) = e^{At}x_0 + \int_0^t e^{A(t-\tau)}Bu(\tau)d\tau
$

In [64]:
A = sympy.Matrix([[0, 1], [-5, -6]])
B = sympy.Matrix([0,1])
x0 = sympy.Matrix([2,0])
sympy.var('T')
t = sympy.var('t')
eat = sympy.exp(A*t)
eat_int = sympy.exp(A*(t-T))
result1 = eat*x0
display(result1)
display(eat*B)
display(eat_int*B)
result2 = integrate(eat_int*B, (T,0,t))
display(result2)
display(result1 + result2)

⎡     -t    -5⋅t  ⎤
⎢  5⋅ℯ     ℯ      ⎥
⎢  ───── - ─────  ⎥
⎢    2       2    ⎥
⎢                 ⎥
⎢     -t      -5⋅t⎥
⎢  5⋅ℯ     5⋅ℯ    ⎥
⎢- ───── + ───────⎥
⎣    2        2   ⎦

⎡   -t    -5⋅t  ⎤
⎢  ℯ     ℯ      ⎥
⎢  ─── - ─────  ⎥
⎢   4      4    ⎥
⎢               ⎥
⎢   -t      -5⋅t⎥
⎢  ℯ     5⋅ℯ    ⎥
⎢- ─── + ───────⎥
⎣   4       4   ⎦

⎡            T - t               5⋅T - 5⋅t⎤
⎢5⋅(-T + t)⋅ℯ        5⋅(-T + t)⋅ℯ         ⎥
⎢───────────────── - ─────────────────────⎥
⎢  4⋅(-5⋅T + 5⋅t)        4⋅(-5⋅T + 5⋅t)   ⎥
⎢                                         ⎥
⎢       5⋅T - 5⋅t               T - t     ⎥
⎢    5⋅ℯ            5⋅(-T + t)⋅ℯ          ⎥
⎢    ──────────── - ─────────────────     ⎥
⎣         4           4⋅(-5⋅T + 5⋅t)      ⎦

⎡     -t    -5⋅t⎤
⎢1   ℯ     ℯ    ⎥
⎢─ - ─── + ─────⎥
⎢5    4      20 ⎥
⎢               ⎥
⎢   -t    -5⋅t  ⎥
⎢  ℯ     ℯ      ⎥
⎢  ─── - ─────  ⎥
⎣   4      4    ⎦

⎡       -t      -5⋅t⎤
⎢1   9⋅ℯ     9⋅ℯ    ⎥
⎢─ + ───── - ───────⎥
⎢5     4        20  ⎥
⎢                   ⎥
⎢      -t      -5⋅t ⎥
⎢   9⋅ℯ     9⋅ℯ     ⎥
⎢ - ───── + ─────── ⎥
⎣     4        4    ⎦

## 安定判別
A(状態行列)の固有値の実部が全て負の場合→漸近安定=発散しないで収束する

[代数的重複度と幾何学的重複度について](http://www.akita-pu.ac.jp/system/elect/ins/kusakari/japanese/teaching/Old/LinearAlgebra/2005/note/10/Slide65.html)

## sympyによる固有値及び固有ベクトル

### eigenvals()の結果の意味
$
\{-2:1, 3:1, 5:2\}
$

→ 固有値は-2,3,5  
→ 固有値の隣の`:1`や`:2`は代数的重複度(重複の数)  
↓  
`{固有値:代数的重複度, }`

### eigenvects()の結果の意味  
$
\begin{bmatrix}
\begin{pmatrix}
5, &
2, &
\begin{bmatrix}
    \begin{bmatrix} 1 \\ 1 \\ 1 \\ 0 \end{bmatrix}, 
    \begin{bmatrix} 0 \\ -1 \\ 0 \\ 1 \end{bmatrix}  
\end{bmatrix}
\end{pmatrix}
\end{bmatrix}
$

→ 固有値は5  
→ 固有値の隣2は幾何学的重複度  
→ 一番右が固有ベクトル  
↓  
`{(固有値, 幾何学的重複度, 固有ベクトル),}`  
`(eigenvalue:algebraic multiplicity, [eigenvectors])`

In [51]:
A = sympy.Matrix([[0, 1], [-5, -6]])
t = sympy.var('t')
result = A.eigenvals()
result2 = A.eigenvects()
display(result)
display(result2)

⎡⎛       ⎡⎡-1/5⎤⎤⎞  ⎛       ⎡⎡-1⎤⎤⎞⎤
⎢⎜-5, 1, ⎢⎢    ⎥⎥⎟, ⎜-1, 1, ⎢⎢  ⎥⎥⎟⎥
⎣⎝       ⎣⎣ 1  ⎦⎦⎠  ⎝       ⎣⎣1 ⎦⎦⎠⎦

### 時間応答シミュレーション

In [24]:
A = "0 1;-5 -6"
B = "0;1"
C = "1 0"
D = "0"
sys = ss(A,B,C,D)
print(sys)
step(sys,X0=[2,0],input=1)

A = [[ 0.  1.]
 [-5. -6.]]

B = [[0.]
 [1.]]

C = [[1. 0.]]

D = [[0.]]



(array([2.        , 1.98041274, 1.93140293, 1.8641393 , 1.78630026,
        1.7031245 , 1.61814802, 1.5337209 , 1.45136956, 1.37205047,
        1.29632777, 1.22449747, 1.15667408, 1.09285108, 1.03294286,
        0.97681386, 0.92429851, 0.87521507, 0.82937482, 0.78658839,
        0.74666979, 0.70943901, 0.67472367, 0.64235983, 0.61219243,
        0.58407536, 0.55787136, 0.53345171, 0.51069595, 0.48949141,
        0.46973286, 0.45132205, 0.43416729, 0.41818307, 0.40328964,
        0.38941265, 0.37648279, 0.36443546, 0.35321046, 0.34275166,
        0.3330068 , 0.32392713, 0.31546727, 0.30758491, 0.30024063,
        0.2933977 , 0.2870219 , 0.28108134, 0.27554631, 0.27038913,
        0.265584  , 0.2611069 , 0.25693543, 0.25304872, 0.24942734,
        0.24605317, 0.24290934, 0.23998012, 0.23725087, 0.23470793,
        0.23233859, 0.23013099, 0.22807409, 0.22615761, 0.22437195,
        0.2227082 , 0.22115802, 0.21971366, 0.2183679 , 0.21711401,
        0.21594572, 0.21485718, 0.21384295, 0.21

## 可制御性
可制御性行列(可制御行列)

$
V_c = \begin{bmatrix}B & AB & \ldots & A^{n-1}B \end{bmatrix}
$

## 可観測性
可観測生行列(可観測行列)

$
V_o =\begin{bmatrix}C \\CA \\\vdots\\CA^{n-1}\end{bmatrix}
$

## 座標変換
$
\dot{x}(t)=Ax(t)+Bu(t) \\ 
y(t)=Cx(t)
$
## 対角正準形
行列Aの固有値を$\lambda_1,\lambda_2,\ldots, \lambda_n $,  
対応する固有ベクトルを$v_1, v_2, \ldots, v_n$  
固有値は全て相異とする  
このとき座標変換行列として

$T = \begin{bmatrix} v_1 & v_2 & \cdots & v_n \end{bmatrix}$

In [20]:
A = sympy.Matrix([[0,1],[-5,-6]])
B = sympy.Matrix([[0],[1]])
C = sympy.Matrix([[0,1]])
display(A,B,C)
result = A.eigenvects()
display("固有ベクトル",result)
#対角化
P,D = A.diagonalize() #P:変換行列, D:対角化後のA行列
display("変換行列",P, "対角行列",D)
A_dash = (P**-1) * A * (P)
B_dash = (P**-1) * B
C_dash = C * P
display("A'",A_dash,"B'",B_dash, "C'",C_dash)

⎡0   1 ⎤
⎢      ⎥
⎣-5  -6⎦

⎡0⎤
⎢ ⎥
⎣1⎦

[0  1]

'固有ベクトル'

⎡⎛       ⎡⎡-1/5⎤⎤⎞  ⎛       ⎡⎡-1⎤⎤⎞⎤
⎢⎜-5, 1, ⎢⎢    ⎥⎥⎟, ⎜-1, 1, ⎢⎢  ⎥⎥⎟⎥
⎣⎝       ⎣⎣ 1  ⎦⎦⎠  ⎝       ⎣⎣1 ⎦⎦⎠⎦

...

⎡-5  0 ⎤
⎢      ⎥
⎣0   -1⎦

"B'"

⎡1/4 ⎤
⎢    ⎥
⎣-1/4⎦

"C'"

[5  1]